<a href="https://colab.research.google.com/github/HSV-AI/presentations/blob/master/2021/210127_WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

This notebook is a copy of the LDA notebook in the [Bug Analysis](https://github.com/HSV-AI/bug-analysis/blob/master/notebooks/Topic_Modeling_LDA.ipynb) project.

## Global Imports

In [24]:
from datetime import date
from datetime import time
from datetime import datetime
from time import sleep
from math import ceil
from dateutil import parser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA
import pyLDAvis
import pyLDAvis.sklearn

# Loading Bugzilla Data

In [5]:
df = pd.read_parquet('https://github.com/HSV-AI/bug-analysis/blob/master/data/df-xtext.parquet.gzip?raw=true')

# Weights and Biases Install and Login

This was copied from the WandB [quickstart](https://docs.wandb.ai/quickstart)



In [2]:
%%capture
!pip install wandb -qqq
import wandb

In [33]:
run = wandb.init(project="topic-modeling-lda", reinit=True)

max_df,0.8
min_df,5
_step,12
_runtime,1666
_timestamp,1611808897
data_sparsicity,1.22887
n_components,15
max_iter,5
learning_offset,50
learning_method,online
random_state,0


max_df,▁█
min_df,█▁
_step,▁▂▂▃▃▄▅▅▆▆▇▇█
_runtime,▁▁▁▁▂▃▃▄█████
_timestamp,▁▁▁▁▂▃▃▄█████
data_sparsicity,█▁
n_components,▁▁▁▁
max_iter,▁▁▁▁
learning_offset,▁▁▁▁
random_state,▁▁
score,██▁


# Create a CountVectorizer

Need to figure out what the optimum data sparsity is here. Also need to figure out a bettwer set of stop words.

In [34]:
vectorizer_params = { 'max_df':0.8, 'min_df':5}

wandb.log(vectorizer_params)
tf_vectorizer = CountVectorizer(**vectorizer_params)
tf = tf_vectorizer.fit_transform(df.loc[:,'text'])
tf_feature_names = tf_vectorizer.get_feature_names()
# Materialize the sparse data
dense = tf.todense()

data_sparsicity = ((dense > 0).sum()/dense.size)*100
wandb.log({'data_sparsicity':data_sparsicity})
# Compute Sparsicity = Percentage of Non-Zero cells
print("Data Sparsicity:", data_sparsicity, "%")
print("Stop Word Count:",len(tf_vectorizer.stop_words_))

Data Sparsicity: 1.2288689104602475 %
Stop Word Count: 35588


# LDA

For LDA, check the perplexity. The lower the better.

In [35]:
lda_params = {
    'n_components':15,
    'max_iter':5,
    'learning_offset':50
}
wandb.log(lda_params)

# Add constant params
lda_params['learning_method'] = 'online'
lda_params['random_state'] = 0

# Run LDA
lda = LatentDirichletAllocation(**lda_params).fit(tf)

score = lda.score(tf)
perplexity = lda.perplexity(tf)

wandb.log({
    'score':score,
    'perplexity':perplexity
})
print("\nLog Likelihood: %d ## Perplexity: %d\n" % ((lda.score(tf)), lda.perplexity(tf)))



Log Likelihood: -11152386 ## Perplexity: 502



# Visualize the Results

This section uses several methods to visualize the results of the LDA.

Using PCA, create a 2D Visualizaton of the TF/IDF Matrix.

In [36]:
# Print the top 5 words in each topic
num_words = 5
print('LDA Topics: ')
for topic_idx, topic in enumerate(lda.components_):
    print ("Topic %d:" % (topic_idx), " ".join([tf_feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]))

LDA Topics: 
Topic 0: typesystem xbase xtext rawtypeconformancecomputer intellij
Topic 1: javareconcilingstrategy projectconfigurationmanager equinoxbundle mavenexecutioncontext moduleclassloader
Topic 2: e4 partrenderingengine workbench swt ui
Topic 3: outline formatting editor formatter jvmmodel
Topic 4: org eclipse xtext http from
Topic 5: com google inject injectorimpl elements
Topic 6: ide guava shared jetty awt
Topic 7: line object thread string run
Topic 8: refactoring deltaconverter storage2urimapperjavaimpl deltaprocessor resource
Topic 9: is that and of this
Topic 10: java at eclipse org internal
Topic 11: xtext org java eclipse common
Topic 12: buildmanager builder xbase events xtextbuilder
Topic 13: emf generator mwe2 antlr mwe
Topic 14: may of and https your


In [37]:

X = tf.todense()

pca = PCA(n_components=2).fit(X)
data2D = pca.transform(X)
plt.scatter(data2D[:,0], data2D[:,1])
plt.plot()

wandb.log({'PCA Scatter': plt})

In [38]:
run.finish()

max_df,0.8
min_df,5
_step,4
_runtime,103
_timestamp,1611809276
data_sparsicity,1.22887
n_components,15
max_iter,5
learning_offset,50
score,-11152386.41522
perplexity,502.95997


max_df,▁
min_df,▁
_step,▁▃▅▆█
_runtime,▁▁▂▆█
_timestamp,▁▁▂▆█
data_sparsicity,▁
n_components,▁
max_iter,▁
learning_offset,▁
score,▁
perplexity,▁
